In [1]:
import copy
import json
from elasticsearch.helpers import bulk
from tqdm import tqdm
import os
import re

from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

stoplist = open('/Users/liyuetian/Documents/College/NEU/CS/CS6200/HW_Data/IR_data/AP_DATA/stoplist.txt')

stop_arr = []
for line in stoplist:
    stop_arr.append(line.strip())
stop_arr = stop_arr + ['document', 'discuss', 'report', 'include', 'describe', 'identify', 'cite', 'predict', 'taken']


In [2]:
new_docno_arr = []
with open('./info/new_docno_arr.txt', 'r') as f:
    new_docno_arr = json.load(f)

In [16]:
query_file = open('./info/query_desc.51-100.short.txt', 'r')
qrel_file = open('./info/qrels.adhoc.51-100.AP89.txt', 'r')

qno_list = []

for line in query_file:
    qno_list.append(line.split('.')[0])
    

In [4]:
qrel_dict = {}
for qno in qno_list:
    qrel_dict[qno] = []


In [5]:
for line in qrel_file:
    if line.split()[0] in qrel_dict:
        if line.split()[2] not in qrel_dict[line.split()[0]]:
            qrel_dict[line.split()[0]].append(line.split()[2])

In [6]:
len(qrel_dict['85'])

614

In [7]:
querylist = open('/Users/liyuetian/Documents/College/NEU/CS/CS6200/HW_Data/IR_data/AP_DATA/query_desc.51-100.short.txt')
query_dict = {}
word_arr = []
query_arr = []

for line in querylist:
    if line.strip() != '':
        queryno = re.sub('[^A-Za-z0-9]+', '', line.split()[0])
        
        modified = line[line.find(".")+1:]
        terms = es.indices.analyze(index='hw6', body={'analyzer': 'stopped', 'text': modified})
        for term in terms['tokens']:
            query_arr.append(term['token'])
        for word in query_arr:
            if word not in stop_arr:
                word_arr.append(word)
        query_dict[queryno] = word_arr
        word_arr = []
        query_arr = []
print(query_dict)


{'85': ['alleg', 'measur', 'corrupt', 'public', 'offici', 'government', 'jurisdict', 'worldwid'], '59': ['type', 'weather', 'event', 'directli', 'caus', 'least', 'fatal', 'locat'], '56': ['prime', 'lend', 'rate', 'actual', 'prime', 'rate', 'move'], '71': ['incurs', 'land', 'air', 'water', 'border', 'area', 'countri', 'militari', 'forc', 'second', 'countri', 'guerrilla', 'group', 'base', 'second', 'countri'], '64': ['event', 'result', 'polit', 'motiv', 'hostag', 'take'], '62': ['militari', 'coup', "d'etat", 'attempt', 'success', 'countri'], '93': ['support', 'nation', 'rifl', 'associ', 'nra', 'asset'], '99': ['develop', 'iran', 'contra', 'affair'], '58': ['anticip', 'rail', 'strike', 'ongo', 'rail', 'strike'], '77': ['poach', 'method', 'type', 'wildlif'], '54': ['sign', 'contract', 'preliminari', 'agreement', 'make', 'tent', 'reserv', 'launch', 'commerci', 'satellit'], '87': ['current', 'crimin', 'action', 'offic', 'fail', 'u.', 'financi', 'institut'], '94': ['crime', 'perpetr', 'aid', 

In [8]:

for qno in query_dict:
    query = ''
    for term in query_dict[qno]:
        query += term + ' '
    print(query)
    res = es.search(index='hw6', body={"query": {"match": {"text": query}}, "size": 12316})
    rel_doc_list = []
    for doc in res['hits']['hits']:
        rel_doc_list.append(doc['_id'])
    for doc in new_docno_arr:
        if len(qrel_dict[qno]) == 1000:
            break
        if doc not in rel_doc_list:
            qrel_dict[qno].append(doc)

alleg measur corrupt public offici government jurisdict worldwid 
type weather event directli caus least fatal locat 
prime lend rate actual prime rate move 
incurs land air water border area countri militari forc second countri guerrilla group base second countri 
event result polit motiv hostag take 
militari coup d'etat attempt success countri 
support nation rifl associ nra asset 
develop iran contra affair 
anticip rail strike ongo rail strike 
poach method type wildlif 
sign contract preliminari agreement make tent reserv launch commerci satellit 
current crimin action offic fail u. financi institut 
crime perpetr aid comput 
effort non communist industri state regul transfer high tech good technolog undesir nation 
exist pend invest opec member state downstream oper 
role israel iran contra affair 
comput applic crime solv 
actual studi unsubstanti concern safeti manufactur employe instal worker fine diamet fiber insul product 
mci bell system breakup 
instanc fiber optic techno

In [9]:
len(qrel_dict['85'])

1000

In [10]:
len(qrel_dict['85'])

1000

In [11]:
with open('./info/qrel_dict.txt', 'w') as f:
    json.dump(qrel_dict, f)

In [17]:

qrel_dict_labeled = {}
for query, doc_list in qrel_dict.items():
    qrel_dict_labeled[query] = {}
    for doc in doc_list:
        qrel_dict_labeled[query][doc] = 0


In [18]:
qrel_file = open('./info/qrels.adhoc.51-100.AP89.txt', 'r')

for line in qrel_file:
    if line.split()[0] in qrel_dict_labeled:
        qrel_dict_labeled[line.split()[0]][line.split()[2]] = line.split()[3]
with open('./info/qrel_dict_labeled.txt', 'w') as f:
    json.dump(qrel_dict_labeled, f)